In [1]:
using LinearAlgebra
using Plots
using Distributions
using Random
using StateSpaceDynamics
using StateSpaceDynamics
using StatsBase

┌ Warning: backend `GR` is not installed.
└ @ Plots C:\Users\ryansenne\.julia\packages\Plots\sxUvK\src\backends.jl:43


In [2]:
# generate data from a Gaussian HMM model
Random.seed!(1234)

A = [0.7 0.2 0.1; 0.1 0.7 0.2; 0.2 0.1 0.7]
    means = [[0.0, 0.0], [-1.0, 2.0], [3.0, 2.5]]
    covs = [
                [0.1 0.0; 0.0 0.1],  # Covariance matrix for state 1
                [0.1 0.0; 0.0 0.1],  # Covariance matrix for state 2
                [0.1 0.0; 0.0 0.1]   # Covariance matrix for state 3
            ]
emissions_models = [SSM.GaussianEmission(mean, cov) for (mean, cov) in zip(means, covs)]
simul_hmm = GaussianHMM(A, emissions_models, [0.33, 0.33, 0.34], 3, 2)
states, observations = SSM.sample(simul_hmm, 10000)


model = GaussianHMM(observations, 3)
# baumWelch!(model, observations, 100)

GaussianHMM{GaussianEmission}([0.393036279038183 0.29923606596931557 0.3077276549925015; 0.31163811901523064 0.03922960829355561 0.6491322726912138; 0.23192224314027649 0.24154571553408974 0.5265320413256337], GaussianEmission[GaussianEmission([3.0001118025261855, 2.494550604933477], [0.0983590175106917 0.001003383195586218; 0.001003383195586218 0.10189301091124765]), GaussianEmission([-1.005304922575402, 2.006852950943985], [0.10189581058001895 0.0017814564889539277; 0.0017814564889539277 0.09168149035717334]), GaussianEmission([-0.005503464515657299, 0.007924397561911023], [0.10161179310002165 -0.006881207595734894; -0.006881207595734894 0.11453015990285093])], [0.038092669321916225, 0.5023725095293903, 0.45953482114869343], 3, 2)

In [4]:
γ, ξ, α, β = SSM.E_step(model, observations)

([-70.59025278753325 -35.41828885774521 0.0; -77.71139517184201 -1.339321897830814e-7 -15.825921268522507; … ; -72.70740127447061 -24.072479314520024 -3.637978807091713e-11; -70.33762368103271 -34.88406840641983 0.0], [-147.9883221512755 -111.01661745110323 -77.71139517184201; -77.71139517184565 -1.339321897830814e-7 -15.825921268526145; … ; -139.07891507795284 -94.97855132335462 -72.70740127467798; -141.98042815925874 -94.32398683147767 -70.33762368107273;;; -70.59025285071766 -35.418289874418406 -1.339321897830814e-7; -161.47192697217906 -85.56027365813861 -99.27312733150757; … ; -90.7573188551687 -48.456696824447135 -24.07247931454185; -106.84019862196146 -60.983499018057046 -34.88406840642347;;; -87.16744504420058 -49.21725971185879 -15.825921268522507; -155.77365505628768 -77.0837793862047 -92.82358435672359; … ; -67.43611059974864 -22.357266212980903 -2.2919266484677792e-10; -72.70740127446697 -24.072479314516386 -3.637978807091713e-11], [-71.85141537499564 -34.647651393823935 -1

In [7]:
sum(exp.(ξ[1, :, :]))

1.000000000001475

In [3]:
α = SSM.forward(model, observations)
β = SSM.backward(model, observations)
γ = SSM.calculate_γ(model, α, β)
ξ = SSM.calculate_ξ(model, α, β, observations)


UndefVarError: UndefVarError: `SSM` not defined

In [2]:
using CSV
using DataFrames

data = CSV.read("/Users/ryansenne/Desktop/observations.csv", DataFrame)
data = Matrix(data)

10000×2 Matrix{Float64}:
  0.343806    -0.132686
 -0.867106     1.78317
  3.10274      2.40358
  2.7328       2.7804
  0.03341     -0.171125
 -0.29612     -0.195364
  3.27012      2.54808
  3.25756      2.59439
  3.10174      2.5855
  2.8045       2.03547
  ⋮           
  0.208154    -0.0825443
 -0.0929328   -0.171961
  0.0773908   -0.178669
  0.0732348    0.107437
  2.60715      2.78216
  0.411477    -0.141309
  0.0822067    0.287452
  0.00112552   0.170238
  0.325549    -0.159369

In [11]:
hmm = GaussianHMM(data, 3)

hmm.A = [0.7 0.2 0.1; 0.2 0.7 0.1; 0.1 0.2 0.7]
hmm.πₖ = [0.3, 0.3, 0.4]
hmm.B = [SSM.GaussianEmission([-1., 2], [1. 0.8; 0.8 1]),
         SSM.GaussianEmission([0.2, 0.2], [1. 0.8; 0.8 1]),
         SSM.GaussianEmission([3, 2.5], [1 0.8; 0.8 1])]

3-element Vector{GaussianEmission}:
 GaussianEmission([-1.0, 2.0], [1.0 0.8; 0.8 1.0])
 GaussianEmission([0.2, 0.2], [1.0 0.8; 0.8 1.0])
 GaussianEmission([3.0, 2.5], [1.0 0.8; 0.8 1.0])

In [12]:
γ, ξ, α, _ = SSM.E_step(hmm, data[1:100, :])

([-13.671072661958277 -0.018102870401605742 -4.020786953139151; -0.0006132961984235408 -7.398774385441811 -13.714950496905288; … ; -11.722206952397698 -0.00021379450734571037 -8.48928630511952; -11.778215492118193 -0.0016180321722742974 -6.432108832157894], [-13.671123294498756 -0.018720700857841166 -4.0211520894868045; -19.68593397670628 -28.356750662884394 -37.2851790170198; … ; -17.53641510270714 -11.725548124668478 -19.67763288017747; -20.999210379405753 -11.778661299857987 -19.7450695683726;;; -23.5670895470453 -7.409161016413606 -11.971208192977997; -4.824347171396653 -10.989637920584045 -20.47768206265488; … ; -8.317052187970319 -0.0006592729409646836 -8.512359816385384; -11.727899349384643 -0.0018243328461835517 -8.527848389296196;;; -28.847886158545208 -13.942720596408918 -15.30609465542264; -0.00868249050543568 -7.426736208188203 -13.716107232708339; … ; -15.558682250736098 -8.495052304202062 -13.80807973009584; -16.910294186825666 -6.436982138782554 -11.764333077681925], [-1

In [15]:
SSM.baumWelch!(hmm, data[1:100, :], 1)

1-element Vector{Any}:
 -243.09082112029844

In [3]:
A

3×2 Matrix{Float64}:
 0.325977  0.894245
 0.549051  0.353112
 0.218587  0.394255

In [3]:
using Random
Random.seed!(1234)
A = rand(3, 2)

SSM.interleave_reshape(A, 3, 2)

Matrix{Float64} (alias for Array{Float64, 2})